In [1]:
from numpy import array
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding

In [2]:
import pandas as pd

df = pd.read_csv("Fake.csv")
df.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [9]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(df["text"][:500])
encoded_data = tokenizer.texts_to_sequences(df["text"][:500])
vocab_size = len(tokenizer.word_counts)+1

In [10]:
vocab_size

15314

In [11]:
seq_list = [seq[:ix] for seq in encoded_data[:500] for ix in range(2, len(seq)+1)]
        
max_length = max([len(seq) for seq in encoded_data])
sequences = pad_sequences(seq_list, maxlen=max_length, padding='pre')
sequences = array(sequences)

In [12]:
X = sequences[:, :-1]
y = sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

In [14]:
model = Sequential([
    Embedding(vocab_size, 10, input_length=max_length-1),
    LSTM(50),
    Dropout(0.1),
    Dense(vocab_size, activation="softmax"),
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X, y, epochs=100)

Epoch 1/100
6647/6647 [==============================] - 340s 51ms/step - loss: 7.1112 - accuracy: 0.0535
Epoch 2/100
6647/6647 [==============================] - 342s 51ms/step - loss: 6.5366 - accuracy: 0.0915
Epoch 3/100
6647/6647 [==============================] - 350s 53ms/step - loss: 6.1759 - accuracy: 0.1191
Epoch 4/100
6647/6647 [==============================] - 334s 50ms/step - loss: 5.9742 - accuracy: 0.1316
Epoch 5/100
6647/6647 [==============================] - 319s 48ms/step - loss: 5.8299 - accuracy: 0.1392
Epoch 6/100
6647/6647 [==============================] - 318s 48ms/step - loss: 5.7036 - accuracy: 0.1473
Epoch 7/100
6647/6647 [==============================] - 322s 48ms/step - loss: 5.5993 - accuracy: 0.1536
Epoch 8/100
6647/6647 [==============================] - 325s 49ms/step - loss: 5.5059 - accuracy: 0.1587
Epoch 9/100
6647/6647 [==============================] - 323s 49ms/step - loss: 5.4240 - accuracy: 0.1651
Epoch 10/100
6647/6647 [======================

KeyboardInterrupt: 

In [17]:
import pickle

with open("model.pkl", "w") as f:
    pickle.dump(model, f)

TypeError: can't pickle _thread.RLock objects

In [25]:
index_to_word = {ix: word for word, ix in tokenizer.word_index.items()}

def predict_sequence(text, num_words):
    for id in range(num_words):
        encoded_data = tokenizer.texts_to_sequences([text])[0]
        padded_data = pad_sequences([encoded_data], maxlen=max_length-1, padding='pre')
        y_pred = model.predict_classes(padded_data)
        print(y_pred)
        out_word = index_to_word[y_pred[0]]
        text += ' ' + out_word
    return text


In [26]:
predict_sequence("Today in Florida", 10)

[4]
[1]
[65]
[89]
[4]
[1]
[65]
[89]
[4]
[1]


'Today in Florida and the white house and the white house and the'

In [27]:
predict_sequence("today there was bipartisan support for", 20)

[1]
[311]
[4]
[1]
[311]
[4]
[1]
[48]
[3]
[1]
[191]
[11]
[16]
[5]
[329]
[17]
[1]
[48]
[3]
[1]


'today there was bipartisan support for the fbi and the fbi and the president of the election he was a war with the president of the'